# Functions for Processing
----

#### Note


In [70]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json as js
from scipy.stats import linregress
from collections import Counter 

# Import API key - usng CBS keys - not used yet
#from api_keys import sport_key
from config import gkey

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Input test file (JSON)
input_data_file='events.json'
# Output File (CSV)
output_data_file = "eventsMaster.csv"


## Generate Events listing

In [71]:
with open(input_data_file) as f:
  data = js.load(f)
#gives a dictionary
#data.values()


In [72]:
import pprint
#pp = pprint.PrettyPrinter(depth=4)
#pp.pprint(data)


Need: 
'count'  - gives the number of events in the list
nested in 'count'
'events' - nested
 by 'contenttype'
     'eventstate' 'scheduled'
     'eventstatus' 'live'
     'eventtype' 'game'
     'is_passthrough' 'False'
     'prismid': '27a62c10-4a15-42ae-a81b-9b31c346ffb9',  [unique id]
     'schedule': {'endtimestamp': 1618181100,
                          'starttimebuffer': 0,
                          'starttimestamp': 1618163100},
     'school': 'nwst',
     'school_name': 'Northwestern State University',
     'sport': 'm-basebl',
     'sport_name': 'Baseball',



### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [73]:
#another way to print

#pp.pprint(f'Dictionary comprehension: {data}')

In [74]:
data.keys()

dict_keys(['events', 'count'])

In [75]:
newDict={}
print(data['count'])
newCount=0
#filter out the passthrough records
Counts=data['count']
#first look for passthrough = True
for index in range(1,Counts):
    if data['events'][index]['is_passthrough']==False:
        newDict=data
        newCount= newCount+1
    
#print(newDict)    
print(f'filtered data counts {newCount}')

706
filtered data counts 558


In [76]:
ID=[]
Type=[]
Scheduled=[]
Status=[]
CType=[]
Pass=[]
Start=[]
End=[]
Event_Title=[]
School_Name=[]
School_Code=[]
Game=[]
PubPoint=[]


for index in range(1,newCount):
    try:
        ID.append(newDict['events'][index]['prismid'])
        #Type.append(newDict['events'][index]['eventtype'])
        Scheduled.append(newDict['events'][index]['eventstate'])
        #Status.append(newDict['events'][index]['eventstatus'])
        #CType.append(newDict['events'][index]['contenttype'])
        Pass.append(newDict['events'][index]['is_passthrough'])
        Start.append(newDict['events'][index]['starttime'])
        End.append(newDict['events'][index]['endtime'])
        Event_Title.append(newDict['events'][index]['eventtitle'])
        School_Name.append(newDict['events'][index]['school_name'])
        School_Code.append(newDict['events'][index]['school'])
        Game.append(newDict['events'][index]['sport_name'])
        #PubPoint.append(newDict['events'][index]['ingest']['primary']['pub_point'])
    except ValueError:
        continue
    except KeyError:
        print(index)
        continue
             
        
  
    
event_df=pd.DataFrame(ID)
#event_df['Event type']=Type
event_df['Scheduled']=Scheduled
#event_df['Is Live']=Status
#event_df['Content Type']=CType
event_df['PassThru']=Pass
event_df['Start Time']=Start
event_df['End Time']=End
event_df['Event']=Event_Title
event_df['School Name']=School_Name
event_df['School Code']=School_Code
event_df['Sport']=Game
#event_df['Access Point']=PubPoint
event_df.rename(columns={0:'ID'},inplace=True)
event_df.set_index('ID',inplace=True)



In [78]:
event_df.head()

,Scheduled,PassThru,Start Time,End Time,Event,School Name,School Code,Sport
ID,,,,,,,,
80454be6-1828-499d-b398-6c3b38f30a28,scheduled,False,1618008600,1618020000,SB: Middle Tennessee vs Marshall,Conference USA,c-usa,Softball
f9547332-f5d5-49e0-bc8f-63ec97466837,scheduled,False,1618073400,1618084800,vs. Ashland,Davenport University,dave,Baseball
cc6a38ff-0b50-4d79-b47f-320b77188954,scheduled,False,1618076700,1618110000,UWG Baseball vs. Union,University of West Georgia,wega,Baseball
bc057b3e-4358-4089-aea2-05ac0004396c,scheduled,False,1617822000,1617840000,Bethany at W&J,Presidents Athletic Conference,pac,Softball
1281a5f2-414e-4349-ab70-63900958ec47,scheduled,False,1618008600,1618020000,BSB AUDIO: Charlotte at FIU,Conference USA,c-usa,Baseball


In [50]:
event_df.columns

Index(['Scheduled', 'PassThru', 'Start Time', 'End Time', 'Event',
       'School Name', 'School Code', 'Sport'],
      dtype='object')

In [51]:
#output dataframe to CSV file
event_df.to_csv('events.csv')

In [52]:
#get the type and pub point values for each prismid
with open(input_data_file) as f:
  filter_set = js.load(f)
#gives a dictionary
filter_set.values()

prismid=[]
Pass=[]
contenttype=[]
pubPoint=[]
#filter out the passthrough records
Counters=filter_set['count']

pr_count=0
content_count=0
pass_count=0
pub_count=0
#first look for passthrough = True
for index in range(1,Counts):
   try:     
    
    if filter_set['events'][index]['is_passthrough']==True:
        Pass.append(filter_set['events'][index]['is_passthrough'])
        prismid.append(filter_set['events'][index]['prismid'])
        contenttype.append(filter_set['events'][index]['contenttype'])
        pubPoint.append(filter_set['events'][index]['ingest']['primary']['pub_point'])
   except KeyError:
        continue

count_pid=len(prismid)
print(count_pid)        
#count_contenttype=len(contenttype)  
#count_pass=len(Pass)
#print(count_pass)
#print(f'pass_throughs are :{Pass}')  
count_pubs=len(pubPoint)
print(count_pubs)
#print(f'pubs are: {pubPoint}')
  
#print(f'prismids to exclude are :{prismid}')

147
0


In [69]:
counter=0
xcount=0
prID=[]
PP=[]
# the pub points
for index in range(1,newCount):
    if newDict['events'][index]['prismid'] in prismid:
        xcount=+1
    elif newDict['events'][index]['prismid']not in prismid:
        prID.append(newDict['events'][index]['prismid'])
        PP.append(newDict['events'][index]['ingest']['primary']['pub_point'])
        
print(xcount)
print(len(prID))  
print(len(PP))

AddPP_df=pd.DataFrame(prID)
AddPP_df['PubPoint']=PP
AddPP_df.rename(columns={0:'ID'},inplace=True)
AddPP_df.set_index('ID',inplace=True)

1
442
442


In [68]:
print(AddPP_df)

                                                  PubPoint
ID                                                        
80454be6-1828-499d-b398-6c3b38f30a28         mtsu_softball
f9547332-f5d5-49e0-bc8f-63ec97466837           davenport_1
cc6a38ff-0b50-4d79-b47f-320b77188954    westgeorgia_audio2
bc057b3e-4358-4089-aea2-05ac0004396c            washjeff_1
1281a5f2-414e-4349-ab70-63900958ec47      charlotte_audio2
...                                                    ...
584cb036-66a8-4a6f-b725-067ffe6c09e0  southalabama_3_audio
a65b3462-a775-445e-b7d2-c330c4476fbe          providence_1
d986f7b6-fea3-4a03-ac1e-8dbfaef364af                 ecu_2
c5127d37-1fa6-418a-aefd-1d1c6a61adcd         cumberlands_2
823a3c4b-c307-4cc0-ad0c-985461829089           villanova_1

[442 rows x 1 columns]


In [92]:
#gets the 558 with 442 pub points
working_events=event_df.merge(AddPP_df,how='left', left_on='ID',right_on='ID')

working_events.tail()
working_events.count()

Scheduled      557
PassThru       557
Start Time     557
End Time       557
Event          557
School Name    557
School Code    557
Sport          557
PubPoint       442
dtype: int64

In [91]:
#output dataframe to CSV file - passthroughs accunted for - audio shown in pubPoint
working_events.to_csv('NewPubPoints_in_events.csv')

In [ ]:
#process for location lat/lng by School Name
#from api_keys import g_key

#school locations to come later this week